# 1.Interactive tissue cut using livewire lasso

This notebook demonstrates the tissue cut process, in which we filter out spots/cells outside of the tissue area.

See our breif tutorial video here: https://www.dropbox.com/s/oy87fpazivg85iw/livewire_tissue_cut.mp4?dl=0

Mouse-keyboard mapping
1. Left mouse button single click: plot point
2. Mouse movement: draw lines attached to tissue boundary
3. Keyboard 'z': draw straight lines instead
4. Keyboard 'Ctrl+C': cancel last operation
5. Repeat above operations and plot on the starting point to end drawing.

## Packages

In [ ]:
import spateo as st
import numpy as np

import matplotlib.pyplot as plt

## Data source

**bin50_h5ad**: https://www.dropbox.com/s/vtapwsccpi885l2/mousebrain_bin50_raw.h5ad?dl=0


In [ ]:
# raw binning data, which contains spot/bins from the whole chip
fname_bin50 = "mousebrain_bin50_raw.h5ad"
adata_bin50 = st.sample_data.mousebrain(fname_bin50)

## Interactive tissue cut

In [ ]:
# rescale spatial coordinates to generate UMI image
coor = np.column_stack((adata_bin50.obsm['spatial'], adata_bin50.X.sum(axis=1).A)).astype('int')
coor[:, 0] = ((coor[:, 0] - 25) / 50).astype('int')
coor[:, 1] = ((coor[:, 1] - 25) / 50).astype('int')
img = np.zeros(shape=[max(coor[:, 0]) + 1, max(coor[:, 1]) + 1], dtype='int')
for line in coor:
    img[line[0], line[1]] = 20 if line[2] > 2000 else int(line[2] / 100)

In [ ]:
# perform interactive tissue cut (interactive process, see tutorial video)
tissue_cut = st.pp.tissue_cut.LiveWireSegmentation(img, smooth_image=False, threshold_gradient_image=False)
plt.scatter(coor[:, 1], coor[:, 0], c=coor[:, 2], s=30)
tissue_cut.connect()
plt.show()

In [ ]:
# subset adata to tissue area only
tissue_area = [x.tolist() in tissue_cut.rst.tolist() for x in adata_bin50.obsm['spatial'].astype('int')]
adata_bin50 = adata_bin50[tissue_area,: ]

adata_bin50